In [15]:
import pandas as pd
pd.options.display.max_columns = None
import numpy as np

In [16]:
df = pd.read_csv('data_traffic_road_segments.csv', index_col=False)
df = df[df['Road name'] == 'N1']
df = df.drop(df.columns[[0]], axis=1)
df = df.rename(columns = {'Start LRP':'LRPName'})
df.drop_duplicates(subset = 'LRPName', keep = 'last', inplace = True)
listLRP = df['LRPName'].unique()
trafficmodes = list(df)[12:-1]
for mode in trafficmodes:
    df[mode + ' Difference'] = df[mode] - df[mode].shift(1)
df.iloc[0, 28:43] = df.iloc[0, 12:27].tolist()
trafficdifference = list(df)[-15:-1]
absolutevalues = list()
for traffic in trafficdifference:
    absolutevalues.append(abs(df[traffic].iloc[0]) )
df.iloc[[0],-15:-1] = absolutevalues
for difference, mode in zip(trafficdifference,trafficmodes):
    df[difference] = np.where(df[difference] < 0, -(1 - (df[mode] / df[mode].shift(1))), df[difference])
    df[mode + ' Create'] = np.where(df[difference] > 0, df[difference], 0) 
    df[mode + ' Destroy'] = np.where(df[difference] < 0, abs(df[difference]), 0) 
CreateDestroy = list(df)[-28:]
df2 = pd.read_csv('data_clean_N1.csv')
df2 = df2.drop(df2.columns[[0,1]], axis=1)
for column in CreateDestroy:
    df2[column] = df2.LRPName.map(df.set_index('LRPName')[column].to_dict())
df2['ControlTraffic'] = np.where(df2['LRPName'].isin(listLRP), 'ControlTraffic', '')
columns = list(df2)[0:-1]
columns.insert(2, list(df2)[-1:][0])
df2 = df2.reindex(columns, axis = 1)
roundcols = list(df2)[10:]
scale = 1
for col in roundcols:
    df2[col] = np.where(df2['ControlTraffic'] == 'ControlTraffic', round((df2[col]/scale),5),np.NaN)
df2['Add On Process'] = np.where(df2['Add On Process'] == 'BridgesDelay', 'BridgesDelay', '')
df2.Cat = df2.Cat.replace('0', '')
df2=df2.rename(columns = {'Add On Process':'BridgesDelay'})
df2['Index'] = df2.index + 1
for row in range(0, len(df2)):
    if df2.ControlTraffic.iloc[row] == 'ControlTraffic':
        df2.BridgesDelay.iloc[row] = 'DestroyTraffic'

In [17]:
df2.head(10)

,LRPName,Lat,ControlTraffic,Lon,Chainage,Description,Length,Cat,Next Node,BridgesDelay,Heavy Truck Create,Heavy Truck Destroy,Medium Truck Create,Medium Truck Destroy,Small Truck Create,Small Truck Destroy,Large Bus Create,Large Bus Destroy,Medium Bus Create,Medium Bus Destroy,Micro Bus Create,Micro Bus Destroy,Utility Create,Utility Destroy,Car Create,Car Destroy,Auto Rickshaw Create,Auto Rickshaw Destroy,Motor Cycle Create,Motor Cycle Destroy,Bi-Cycle Create,Bi-Cycle Destroy,Cycle Rickshaw Create,Cycle Rickshaw Destroy,Cart Create,Cart Destroy,Motorized Total Create,Motorized Total Destroy,Index
0,LRPS,23.706028,ControlTraffic,90.443333,0.000,0,0.0,,LRPSa,DestroyTraffic,28.10267,0.0,503.61365,0.0,251.46306,0.0,430.90765,0.0,241.6658,0.0,233.24359,0.0,57.1507,0.0,303.11354,0.0,502.49641,0.0,198.09376,0.0,62.56497,0.0,224.13385,0.0,0.0,0.0,2749.85083,0.0,1
1,LRPSa,23.702917,,90.450417,0.814,0,0.0,,LRPSb,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
2,LRPSb,23.702778,,90.450472,0.822,0,0.0,,LRP001,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
3,LRP001,23.702139,,90.451972,1.000,0,0.0,,LRP001a,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
4,LRP001a,23.698739,,90.458861,1.800,.,11.3,A,LRP002,BridgesDelay,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5
5,LRP002,23.697889,,90.460583,2.000,0,0.0,,LRP002a,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6
6,LRP002a,23.697361,,90.461667,2.130,0,0.0,,LRP003,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7
7,LRP003,23.693833,,90.469138,3.000,0,0.0,,LRP004,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8
8,LRP004,23.693611,,90.478777,4.000,0,0.0,,LRP004a,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
9,LRP004a,23.693805,,90.480527,4.175,0,0.0,,LRP004b,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10


In [18]:
df2[df2['ControlTraffic'] == 'ControlTraffic'].iloc[0:9]

,LRPName,Lat,ControlTraffic,Lon,Chainage,Description,Length,Cat,Next Node,BridgesDelay,Heavy Truck Create,Heavy Truck Destroy,Medium Truck Create,Medium Truck Destroy,Small Truck Create,Small Truck Destroy,Large Bus Create,Large Bus Destroy,Medium Bus Create,Medium Bus Destroy,Micro Bus Create,Micro Bus Destroy,Utility Create,Utility Destroy,Car Create,Car Destroy,Auto Rickshaw Create,Auto Rickshaw Destroy,Motor Cycle Create,Motor Cycle Destroy,Bi-Cycle Create,Bi-Cycle Destroy,Cycle Rickshaw Create,Cycle Rickshaw Destroy,Cart Create,Cart Destroy,Motorized Total Create,Motorized Total Destroy,Index
0,LRPS,23.706028,ControlTraffic,90.443333,0.000,0,0.0,,LRPSa,DestroyTraffic,28.10267,0.00000,503.61365,0.00000,251.46306,0.00000,430.90765,0.00000,241.66580,0.00000,233.24359,0.00000,57.15070,0.00000,303.11354,0.00000,502.49641,0.00000,198.09376,0.00000,62.56497,0.00000,224.13385,0.00000,0.0,0.0,2749.85083,0.00000,1
18,LRP009,23.705028,ControlTraffic,90.519333,8.503,0,0.0,,LRP009a,DestroyTraffic,9.07026,0.00000,563.14732,0.00000,510.07058,0.00000,271.17203,0.00000,349.34979,0.00000,282.63037,0.00000,10.37446,0.00000,115.67881,0.00000,206.17663,0.00000,63.02622,0.00000,20.19313,0.00000,72.34021,0.00000,0.0,0.0,2380.69647,0.00000,19
27,LRP012,23.691805,ControlTraffic,90.542611,11.497,0,0.0,,LRP012a,DestroyTraffic,2.81217,0.00000,80.70161,0.00000,57.61084,0.00000,53.11308,0.00000,44.71096,0.00000,39.02642,0.00000,5.10835,0.00000,31.68209,0.00000,53.61188,0.00000,19.75401,0.00000,6.26074,0.00000,22.42858,0.00000,0.0,0.0,388.13140,0.00000,28
32,LRP013,23.686527,ControlTraffic,90.550417,12.524,0,0.0,,LRP013a,DestroyTraffic,30.20397,0.00000,60.98483,0.00000,0.00000,0.69659,0.00000,0.46935,0.00000,0.74011,0.00000,0.30103,20.99100,0.00000,29.71731,0.00000,0.00000,0.61816,0.00000,0.82311,0.00000,0.91049,0.00000,0.87985,0.0,0.0,0.00000,0.38472,33
61,LRP022,23.625222,ControlTraffic,90.604972,21.629,0,0.0,,LRP022a,DestroyTraffic,4.68592,0.00000,80.67760,0.00000,16.59238,0.00000,26.75433,0.00000,11.03029,0.00000,25.89381,0.00000,6.25050,0.00000,32.05826,0.00000,19.43209,0.00000,3.31691,0.00000,0.53196,0.00000,2.55809,0.00000,0.0,0.0,226.69208,0.00000,62
89,LRP031,23.559972,ControlTraffic,90.654806,30.774,0,0.0,,LRP031a,DestroyTraffic,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.0,0.0,0.00000,0.00000,90
96,LRP033,23.548750,ControlTraffic,90.667833,32.607,0,0.0,,LRP033b,DestroyTraffic,0.00000,0.06473,0.00000,0.25370,0.00000,0.33537,0.00000,0.24375,0.00000,0.32674,0.00000,0.66628,0.00000,0.21157,0.00000,0.57944,0.00000,0.49406,0.00000,0.38327,0.00000,0.18430,0.00000,0.38692,0.0,0.0,0.00000,0.37268,97
128,LRP043,23.529416,ControlTraffic,90.751750,42.269,0,0.0,,LRP043a,DestroyTraffic,0.00000,0.15867,0.00000,0.15867,0.00000,0.15867,0.00000,0.15867,0.00000,0.15867,0.00000,0.15867,0.00000,0.15867,0.00000,0.15867,0.00000,0.15867,0.00000,0.15867,0.00000,0.15867,0.00000,0.15867,0.0,0.0,0.00000,0.15867,129
179,LRP067,23.497277,ControlTraffic,90.967472,65.726,0,0.0,,LRP067a,DestroyTraffic,0.00000,0.08911,95.41667,0.00000,0.00000,0.18887,35.25000,0.00000,0.00000,0.22538,50.16667,0.00000,1.08333,0.00000,6.83333,0.00000,193.08333,0.00000,64.91667,0.00000,12.83333,0.00000,8.75000,0.00000,0.0,0.0,391.00000,0.00000,180


In [207]:
#df2.to_csv('AutoCreateSimioAssignment4.csv')
#df2.to_excel('AutoCreateSimioAssignment4.xlsx')